In [1]:
from PIL import Image, ImageEnhance
import os
import random
from matplotlib.pyplot import imshow
import numpy as np
%matplotlib inline

# from google.colab import drive
# drive.mount('/content/drive')

# Binary image random pixel removal

In [3]:
# image_file = Image.open("D:/RESEARCH/Circuit component recognition/train/6/componentl016.bmp")

def binary_augment(image_file):
    image_file = image_file.convert('L') # binary
    image_file = image_file.point(lambda p: 255 if p < random.randint(100,110) else p)
    image_file = image_file.convert('RGB') # mono
    return image_file

# image = binary_augment(image_file)
# # image.save("trial.bmp")
# imshow(binary_augment(image_file))

# Brightness

In [4]:
# image_file = Image.open("/content/drive/MyDrive/data/train/5/componentl001.bmp")
def brightness(image_file):
    image_file = ImageEnhance.Brightness(image_file)
    return image_file.enhance(1.5)

# imshow((brightness(image_file)))

# Random rotation (-20 to -10) or (10 to 20)

In [5]:
# img = Image.open("/content/drive/MyDrive/data/train/2/componentl001.bmp")
def rotator(img):
  decider = random.randint(1,2)
  if decider == 1:
    img = img.rotate(random.randint(10,20))
  else:
    img = img.rotate(random.randint(-20,-10)) 
  return img 

# imshow(rotator(img)) 

# Contrast

In [6]:
from PIL import Image, ImageEnhance

#read the image
# im = Image.open("/content/drive/MyDrive/data/train/5/componentl001.bmp")
def contraster(im):
  #image brightness enhancer
  enhancer = ImageEnhance.Contrast(im)
  return enhancer.enhance(1.75)

# imshow(contraster(im))  

# Dilation based thickening

In [7]:
import cv2
# img = cv2.imread("/content/drive/MyDrive/data/train/5/componentl001.bmp", cv2.IMREAD_GRAYSCALE)

def masking(img,i,j,z):
  arr = []
  avg = []
  for k in range(-z,z):
    for l in range(-z,z):
      arr.append(img[i-k][j-l])
  arr.sort()
  for w in range(0,round((len(arr)-1)/2)):
    avg.append(arr[w])
  return (sum(avg)/((len(arr)-1)/2)) #avg of first half after sorting to get the darker values.

def dilation_thick(img):
  org_img = img #original image
  mod_img = img #image to be modified
  img = 255 - img
  (thresh, img) = cv2.threshold(img, 40, 255, cv2.THRESH_BINARY)
  kernel = np.ones((7,7), np.uint8)
  img_dilation = cv2.dilate(img, kernel, iterations=1) #dilated image
  difference = img_dilation-img 
  unique,counts = np.unique(difference,return_counts=True)
  print(dict(zip(unique, counts)))
  for i in range(img.shape[0]): # height
    for j in range(img.shape[1]): # width
      if (difference[i][j]==0):
        mod_img[i][j] = org_img[i][j]
      else:
        try:
          mod_img[i][j] = masking(org_img,i,j,5)
        except:
          mod_img[i][j] = org_img[i][j]
  (thresh, bin_mod) = cv2.threshold(mod_img, 40, 255, cv2.THRESH_BINARY)                   
  return mod_img

# imshow(dilation_thick(img))

# Saving the augmented images

In [11]:
for i in range(10,21):
  folder = "D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.o/train/"+str(i)+"/"
  folder2 = "D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Train_a/"+str(i)+"/"
  for image in os.listdir(os.path.join(folder)):
      #print(image[13:17])
      #if image[13:17] == ".bmp":
          print(os.path.join(folder,image))
          img = Image.open(os.path.join(folder,image))
          img_cv = cv2.imread(os.path.join(folder,image), cv2.IMREAD_GRAYSCALE)
          rotated_pos = img.rotate(random.randint(1,10)) # random positive angle rotation
          rotated_neg = img.rotate(random.randint(-10,-1)) # random negative angle rotation
          bin_img = binary_augment(img)
          bright_img = brightness(img)
          contrast_img = contraster(img)
          rot_img = rotator(img)
          dilation_img = dilation_thick(img_cv)
          #print(folder2+"augmented_03"+image[9:11]+".bmp")
          rotated_pos.save(folder2+"augmented_03_"+image[9:11]+".bmp")
          rotated_neg.save(folder2+"augmented_02_"+image[9:11]+".bmp")
          bin_img.save(folder2+"augmented_09_"+image[9:11]+".bmp")
          bright_img.save(folder2+"augmented_04_"+image[9:11]+".bmp")
          contrast_img.save(folder2+"augmented_05_"+image[9:11]+".bmp")
          rot_img.save(folder2+"augmented_01_"+image[9:11]+".bmp")
          cv2.imwrite(folder2+"augmented_08_"+image[9:11]+".bmp",dilation_img)

D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.o/train/10/original_01.bmp
{0: 24224, 255: 3314}
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.o/train/10/original_02.bmp
{0: 28631, 255: 3244}
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.o/train/10/original_03.bmp
{0: 36823, 255: 4217}
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.o/train/10/original_04.bmp
{0: 32511, 255: 3457}
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.o/train/10/original_05.bmp
{0: 32863, 255: 3363}
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.o/train/10/original_06.bmp
{0: 36568, 255: 3182}
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.o/train/10/original_07.bmp
{0: 27029, 255: 2483}
D:/RESEARCH/Circuit component reco

# Removing black offsets in rotated images

In [41]:
import cv2
for i in range(10,21):
  folder = "D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Test_a/"+str(i)+"/"

  for image in os.listdir(os.path.join(folder)):
    #print(image[:13])
    print(os.path.join(folder,image))
    if image[:13]=="augmented_03_":
      img = cv2.imread(os.path.join(folder,image))
      height, width, _ = img.shape
      for i in range(height):
        for j in range(width):
          #img[i, j] is the RGB pixel at position (i, j)
          #check if it's [0, 0, 0] and replace with [255, 255, 255] if so
          if img[i, j].sum() == 0:
            img[i, j] = [255, 255, 255]
      cv2.imwrite(os.path.join(folder,image),img)                       

D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Test_a/10/augmented_01_00.bmp
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Test_a/10/augmented_01_01.bmp
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Test_a/10/augmented_01_02.bmp
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Test_a/10/augmented_01_03.bmp
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Test_a/10/augmented_01_04.bmp
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Test_a/10/augmented_01_05.bmp
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Test_a/10/augmented_01_06.bmp
D:/RESEARCH/Circuit component recognition/Circuit-Component-Analysis/Dataset/CktComDb-v1.a/Test_a/10/augmented_01_07.bmp
D:/RESEARCH/Circuit component re

In [28]:
j = 0
for i in range(11,12):
  folder = "C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test/"+str(i)+"/"
  for image in os.listdir(os.path.join(folder)):
    img = Image.open(os.path.join(folder,image))
    if j<10:
          print(j)
          img.save("C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/"+str(i)+"/"+"mixed_00"+str(j)+".bmp")
          print("C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/"+str(i)+"/"+"mixed_00"+str(j)+".bmp")
    elif 9<j<100:
          print(j)
          img.save("C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/"+str(i)+"/"+"mixed_0"+str(j)+".bmp")
          print("C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/"+str(i)+"/"+"mixed_0"+str(j)+".bmp")
    else:
          print(j)
          img.save("C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/"+str(i)+"/"+"mixed_"+str(j)+".bmp")
          print("C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/"+str(i)+"/"+"mixed_"+str(j)+".bmp")
    j = j+1

0
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_000.bmp
1
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_001.bmp
2
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_002.bmp
3
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_003.bmp
4
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_004.bmp
5
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_005.bmp
6
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_006.bmp
7
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_007.bmp
8
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_008.bmp
9
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_009.bmp
10
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_010.bmp
11
C:/Users/aroy8/Downloads/CktComDb-v1.oa/CktComDb-v1.oa/Test_a/11/mixed_011.bmp
12
C:/Users/aroy8/Download